In [112]:
from bs4 import BeautifulSoup
import requests
from collections import defaultdict
import pandas as pd
import re
from datetime import datetime
import numpy as np


In [113]:
link_list = []
for i in range(1,50):
    url ='https://www.boxofficemojo.com/alltime/domestic.htm?page={}&p=.htm'.format(i)
    response = requests.get(url)
    page = response.text
    soup = BeautifulSoup(page, "lxml")
    for j in range(4,100): #4 to 100 seems to work
        link_list.append('https://www.boxofficemojo.com' + soup.find('div', id ='body').find_all('table')[3].find_all('tr')[j].find('a').get('href'))
      
        
        
        

In [114]:
mv_info = defaultdict(dict)

**Web Scraper**

In [115]:
def html_cleaner(str2clean):
    if '<a>' in str2clean:
        str2clean = str2clean.replace('<a>','')
    if '</a>' in str2clean:
        str2clean = str2clean.replace('</a>','')
    if '<b>' in str2clean:
        str2clean = str2clean.replace('<b>','')
    if '</b>' in str2clean:
        str2clean = str2clean.replace('</b>','')
    if '<td>' in str2clean:
        str2clean = str2clean.replace('<td>','')
    if '</td>' in str2clean:
        str2clean = str2clean.replace('</td>','')    
    return str2clean


def box_office_mojo_scraper(url,mv_info):
    response = requests.get(url)
    page = response.text
    soup = BeautifulSoup(page, "lxml")
    title = soup.find('title').text.split('(')[0]


    try:
        budget = str(soup.find(text=re.compile('Production Budget')).findNextSibling())
        budget = html_cleaner(budget)
        budget = budget.replace('$','')
        budget = budget.split()
        if budget[1] == 'billion':
            budget[1] = '000000000'
            budget = ''.join(budget)
        if budget[1] == 'million':
            budget[1] = '000000'
            budget = ''.join(budget)
        if budget[1] == 'thousand':
            budget[1] = '000'
            budget = ''.join(budget)
        mv_info[title]['budget'] = int(budget)
    except:
        mv_info[title]['budget'] = np.nan

    #**Pull Genre with RegEx**

    try:
        genre = str(soup.find(text=re.compile('Genre:')).findNextSibling())
        genre = html_cleaner(genre)
        mv_info[title]['genre'] = genre
    except:
        mv_info[title]['genre'] = np.nan

    #**Pull Distributor with RegEx**

    try:
        distributor = str(soup.find(text=re.compile('Distributor:')).findNextSibling())
        distributor = html_cleaner(distributor)
        distributor = distributor.split('>')[1]
        mv_info[title]['distributor'] = distributor
    except:
        mv_info[title]['distributor'] = np.nan

    #**Pull Director**

    try:
        director_orig = str(soup.find_all(class_='mp_box_content')[2].find_all('td')[1])
        director = director_orig.split('>')[3]
        if director ==  '</td':
            director = director_orig.split('>')[2]
            director = director.replace('</font','')
        director = director.replace('</a','')
        mv_info[title]['director'] = director
    except:
        mv_info[title]['director'] = np.nan

    #**Pull Release Date with RegEx**

    try:
        release_date = str(soup.find(text=re.compile('Release Date')).findNextSibling())
        release_date = release_date.split('>')[3]
        release_date = release_date.replace('</a','')
        release_date = datetime.strptime(release_date, '%B %d, %Y')
        release_date = release_date.strftime('%d %m %Y')
        mv_info[title]['release_date'] = release_date
    except:
        mv_info[title]['release_date'] = np.nan
        
     #**Pull close date**   
    try:
        close_date = soup.find_all(class_='mp_box')[1].find(class_='mp_box_content').find_all('tr')[5].find_all('td')[1]
        close_date = html_cleaner(str(close_date))
        close_date = close_date[1:]
        close_date = datetime.strptime(close_date, '%B %d, %Y')
        mv_info[title]['close_date'] = close_date.strftime('%d %m %Y')
    except:
        mv_info[title]['close_date'] = np.nan
    
    #**pull days in release**
    try:
        days_in_release = soup.find_all(class_='mp_box')[1].find(class_='mp_box_content').find_all('table')[3]
        days_in_release = str(days_in_release).split()[8]
        mv_info[title]['days_in_release'] = int(days_in_release)
    
    except:
        mv_info[title]['days_in_release'] = np.nan
        
        

    #**Pull Runtime with RegEx**

    try:
        runtime_orig = str(soup.find(text=re.compile('Runtime')).findNextSibling())    
        runtime_orig = html_cleaner(runtime_orig)
        runtime = runtime_orig.split()
        runtime = int(runtime[0]) * 60 + int(runtime[2])
        mv_info[title]['runtime'] = runtime
    except:
        runtime = np.nan
    

    #**Pull Rating with RegEx**

    try:
        rating = str(soup.find(text=re.compile('MPAA Rating:')).findNextSibling())
        rating = html_cleaner(rating)
        mv_info[title]['rating'] = rating
    except:
        mv_info[title]['rating'] = np.nan

    #**Pull gross domestic from class mp_box_contet**

    try:
        gross_domestic = soup.find(class_='mp_box_content').find_all('td')[1].text.split('$')[1] 
        gross_domestic = int(gross_domestic.replace(',',''))
        mv_info[title]["domestic"] = gross_domestic
    except:
        mv_info[title]["domestic"] = np.nan
      
    #**Pull % gross domestic 
    try:
        prcnt_domstc = soup.find(class_='mp_box_content').find_all('td')[2].text
        prcnt_domstc = prcnt_domstc.split()[0]
        prcnt_domstc = float(prcnt_domstc.replace('%',''))
        mv_info[title]['percent_domestic'] = prcnt_domstc
    except:
        mv_info[title]["percent_domestic"] = np.nan

    #**Pull foreign from class mp_box_content**

    try:    
        frgn = soup.find(class_='mp_box_content').find_all('td')[4].text.split('$')[1]
        frgn = frgn.replace(',','')
        mv_info[title]["foreign"] = int(frgn)
    except:
        mv_info[title]["foreign"] = np.nan
        
    #**Pull % gross foreign
    try:    
        prcnt_frgn = soup.find(class_='mp_box_content').find_all('td')[5].text
        prcnt_frgn = prcnt_frgn.split()[0]
        prcnt_frgn = float(prcnt_frgn.replace('%',''))
        mv_info[title]["percent_foreign"] = prcnt_frgn
    except:
        mv_info[title]["percent_foreign"] = np.nan

    #**Pull Opening Weekend from second mp_box class**

    try:
        open_wknd = soup.find_all(class_='mp_box')[1].find(class_='mp_box_content').find('tr').text.split('$')[1]
        open_wknd = open_wknd.replace(',','')
        mv_info[title]["open_wknd"] = int(open_wknd)
    except:
        mv_info[title]["open_wknd"] = np.nan  
        
        
   #**Pull % total domestic drawn in open wknd
    try:
        prcnt_wknd = soup.find_all(class_='mp_box')[1].find(class_='mp_box_content').find_all('tr')[2].text.split()
        prcnt_wknd = prcnt_wknd[4].replace('%','')
        prcnt_wknd = float(prcnt_wknd)
        mv_info[title]['prcnt_dom_wknd'] = prcnt_wknd
    except:
        mv_info[title]['prcnt_dom_wknd'] = np.nan
        

    #**Pull Theater Rank, # of theaters and avg money per theater from second mp_box class** 

    try:
        thtr_info = soup.find_all(class_='mp_box')[1].find(class_='mp_box_content').find_all('tr')[1].text
        thtr_info = thtr_info.split(" ") #split thtr info into list of strings seperated by ' '

        thtr_rank = int(thtr_info[0].split('#')[1]) #take thtr_info, split at hashtag, take second element, convert to int
        mv_info[title]["rank"] = thtr_rank #add thtr rank to dict

        num_thtr = thtr_info[2] #rank is useless without the number of theaters its ranked relative to, so take that as well
        num_thtr = thtr_info[2].replace(',','')
        mv_info[title]["numbr_thtr"] = int(num_thtr) #add number of theaters to dict

        avg_per_thtr = thtr_info[4].replace(',','',) #remove comma from avg $ per thtr
        avg_per_thtr = int(avg_per_thtr.replace('$','')) #remove $, convert to int
        mv_info[title]["avg_per_thtr"] = avg_per_thtr #add avg_per_thtr to dict

    except:
        mv_info[title]["rank"] = np.nan
        mv_info[title]["numbr_thtr"] = np.nan
        mv_info[title]["avg_per_thtr"] = np.nan

    #**Pull widest release from second mp_box class and 5th table** 

    try:
        widest_release = soup.find_all(class_='mp_box')[1].find(class_='mp_box_content').find_all('tr')[4].text
        widest_release = widest_release.split()[2]
        widest_release = int(widest_release.replace(',',''))
        mv_info[title]["wdst_rel"] = widest_release
    except:
         mv_info[title]["wdst_rel"] = np.nan
            
    return(mv_info)

In [116]:
for i in link_list:
    box_office_mojo_scraper(i, mv_info)

In [117]:
#mv_info

In [118]:
df = pd.DataFrame.from_dict(mv_info)

In [119]:
df = df.transpose()

In [120]:
df_original = df.copy(deep = True)

In [121]:
df.to_csv('movie_data', sep=',')